# Como hemos extraído las preguntas y respuestas del PDF

In [5]:
# Instalando el módulo 
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Enseñando que el módulo se ha instalado correctamente
%pip show PyPDF2


Name: PyPDF2
Version: 3.0.1
Summary: A pure-python PDF library capable of splitting, merging, cropping, and transforming PDF files
Home-page: 
Author: 
Author-email: Mathieu Fenniak <biziqe@mathieu.fenniak.net>
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Paso 1 - importamos los módulos que vamos a usar.
from PyPDF2 import PdfReader
import json # Usamos el módulo json para formatear el diccionario a la hora de guardarlo en un archivo .py


In [26]:
# Paso 2 - Crear el diccionario de la tarea en el archivo .py 

# Abrimos el archivo PDF
with open('Manual_copy_5.pdf', 'rb') as file: #                         >>> Elegir el archivo que quiero extraer
    reader = PdfReader(file)
    
    # Extraer el número de páginas
    num_pages = len(reader.pages)
    
    # Extraer el texto de cada página
    text = ""
    for i in range(num_pages):
        page = reader.pages[i]
        text += page.extract_text()

    preg_resp_diccio = {}
    lista_indices_preg = []
    for i in range(5001, 5086): #                                        >>> Actualizar las páginas según los IDs de las preguntas de la tarea correspondiente (el rango final + 2)
        indice_pregunta = text.find(str(i))
        lista_indices_preg.append(indice_pregunta)

    counter = 0
    perguntas = []
    for i in lista_indices_preg:
        try:
            counter+=1
            indice_final = lista_indices_preg[counter]
            pregunta = text[i:indice_final]
            perguntas.append(pregunta)
        except IndexError as e:
            pass
    
    counter = 0
    for i in perguntas:
        try:
            preg_limpia = i.strip()   
            perguntas[counter] = preg_limpia[5:]
            counter+=1
        except IndexError as e:
            pass
    
    for i in perguntas:
        indice_resp = i.find("a)")
        preguntas = i[:indice_resp]
        respuestas = i[indice_resp:-1].replace(".", ",")
        if respuestas.endswith(", 1") or respuestas.endswith(", 2"):
            respuestas = respuestas[:-3]
        respuestas += ","
        soluciones = perguntas[-1]
        preg_resp_diccio[preguntas] = respuestas.split(",")

    # print(soluciones)

    solucciones = []
    for i in range(5001, 5085): #                                               >>> Actualizar las páginas según los IDs de las preguntas de la tarea correspondiente (el rango final + 1)
        indice_solucc = soluciones.find(str(i))
        solucciones.append(soluciones[indice_solucc+5])

    # print(solucciones)

    counter = 0
    for valor in preg_resp_diccio.values():
        valor[-1] = solucciones[counter]
        counter +=1
            
    # print(preg_resp_diccio)                                                   >>> Diccionario final en formato correcto.

    # Código para guardar el dict en un archivo .py
    file_path = "tarea5.py" #                                                   >>> Al cambiar de archivo hay que cambiar el número de la tarea
    with open(file_path, "w") as file:
        json_str = json.dumps(preg_resp_diccio, indent=4, ensure_ascii=False)
        file.write("tarea5 = " + json_str) #                                    >>> Al cambiar de archivo hay que cambiar el número de la tarea

    # NOTA: Para guardar el archivo sin que esté formateado con json:
    # file_path = "tarea1.py"
    # with open(file_path, "w") as file:
    #     file.write("tarea1 = " + str(preg_resp_diccio))

